<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Kentucky_demo_Data_API_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.0 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

package not found, installing w/ pip in Google Colab...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.9 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11124 sha256=0f25975a8bafc9ab

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135743 sha256=12fa1dbbe6893ed8adaa3504b146b258dc4d691d59caca76abcc2d72480ff4f6
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236757 sha256=68199f95873116c4a2c801aaf92220575cc9e70d367ed1f0ef23a0f6db99d790
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [7]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Gi8RTYeyS1kwgg9y9ANNMOFWARvgfIAelqhCECY90JI&tc=rg_fvtwwesY72mbaDtR3bU6APMXp_IPLVt2VNt-elWs&cc=kSIV82ZtjZ6RsG_7rsfQ-t1uDT1qF4iTTOZ9BtRPML4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhWIBRcHli2G2vmbPy-hu8UrFfjpM1pg3tfEGGNPP4zAw7vNM7tQGXw

Successfully saved authorization token.


In [8]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/KY_Released_mines.geojson')

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))
mines = mines.head(20)

Data dimensions: (12606, 19)


In [9]:
# Get the shape geometry
region = eec.gdfToFc(mines)

In [10]:
df = pd.DataFrame()

In [11]:
mines_exploded= mines.explode()
mines_exploded

<ipython-input-11-86b5dc24b419>:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  mines_exploded= mines.explode()


id  ACT_INAC  Calc_Acres                      Contact  \
0  0  00000000000000001111  Released  294.302954  KY Division of Mine Permits   
1  0  00000000000000001112  Released  305.739886  KY Division of Mine Permits   
   1  00000000000000001112  Released  305.739886  KY Division of Mine Permits   
   2  00000000000000001112  Released  305.739886  KY Division of Mine Permits   
   3  00000000000000001112  Released  305.739886  KY Division of Mine Permits   
2  0  00000000000000001f4d  Released    4.822285  KY Division of Mine Permits   
3  0  00000000000000002b96  Released    6.335894  KY Division of Mine Permits   
4  0  00000000000000001d20  Released    2.864151  KY Division of Mine Permits   
5  0  000000000000000020c7  Released    2.799211  KY Division of Mine Permits   
   1  000000000000000020c7  Released    2.799211  KY Division of Mine Permits   
6  0  00000000000000003095  Released   38.458558  KY Division of Mine Permits   
   1  00000000000000003095  Released   38.458558  KY Division of Mine Permits   
7  0  0000000000000000080c  Released    2.140314  KY Division of Mine Permits   
8  0  0000000000000000086a  Released   65.428833  KY Division of Mine Permits   
9  0  000000000000000008a4  Released   86.624399  KY Division of Mine Permits   
10 0  000000000000000008bf  Released    9.394097  KY Division of Mine Permits   
11 0  00000000000000000f90  Released    2.635152  KY Division of Mine Permits   
12 0  00000000000000000fd6  Released    0.462351  KY Division of Mine Permits   
13 0  0000000000000000201c  Released   36.476694  KY Division of Mine Permits   
   1  0000000000000000201c  Released   36.476694  KY Division of Mine Permits   
   2  0000000000000000201c  Released   36.476694  KY Division of Mine Permits   
14 0  000000000000000003af  Released   10.959218  KY Division of Mine Permits   
15 0  000000000000000014f0  Released   27.882968  KY Division of Mine Permits   
   1  000000000000000014f0  Released   27.882968  KY Division of Mine Permits   
16 0  000000000000000014fc  Released    2.139254  KY Division of Mine Permits   
   1  000000000000000014fc  Released    2.139254  KY Division of Mine Permits   
17 0  00000000000000002811  Released    3.756246  KY Division of Mine Permits   
   1  00000000000000002811  Released    3.756246  KY Division of Mine Permits   
   2  00000000000000002811  Released    3.756246  KY Division of Mine Permits   
18 0  00000000000000002821  Released   78.265871  KY Division of Mine Permits   
   1  00000000000000002821  Released   78.265871  KY Division of Mine Permits   
   2  00000000000000002821  Released   78.265871  KY Division of Mine Permits   
   3  00000000000000002821  Released   78.265871  KY Division of Mine Permits   
   4  00000000000000002821  Released   78.265871  KY Division of Mine Permits   
19 0  0000000000000000283f  Released   49.520246  KY Division of Mine Permits   
   1  0000000000000000283f  Released   49.520246  KY Division of Mine Permits   
   2  0000000000000000283f  Released   49.520246  KY Division of Mine Permits   
   3  0000000000000000283f  Released   49.520246  KY Division of Mine Permits   

           DATE_ISS FeatCLS MINE_STATU National_I ORIG_PERM  \
0  0  1141718400000      SF         FF  KY8640180   8640180   
1  0  1141718400000      SF         FF  KY8640180   8640180   
   1  1141718400000      SF         FF  KY8640180   8640180   
   2  1141718400000      SF         FF  KY8640180   8640180   
   3  1141718400000      SF         FF  KY8640180   8640180   
2  0   374486400000      SF         FF  KY0130119   0130119   
3  0   322732800000      SF         FF  KY1005000   1005000   
4  0   455871600000      SF         FF  KY9180010   9180010   
5  0   388137600000      SF         FF  KY0320080   0320080   
   1   388137600000      SF         FF  KY0320080   0320080   
6  0   172738800000      SF         FF  KY404774X   404774X   
   1   172738800000      SF         FF  KY404774X   404774X   
7  0   463302000000      SF         FF  KY826

In [12]:
# Get the shape geometry
region = eec.gdfToFc(mines_exploded)

In [27]:
i=2

# Initial date of interest (inclusive).
i_date = '2021-01-01'
# Final date of interest (exclusive).
f_date = '2021-01-15'

df_contructor = []
while i<=len(mines_exploded):
        # Get dynamic world image collection by asset ID
        dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date)

        # get band names, equal to column names
        bandnames = dc.first().bandNames().getInfo()[:-1]

        polygon = mines_exploded['geometry'].values[i]

        # Extract the coordinates from the polygon
        coordinates = polygon.exterior.coords[:]

        # Create an ee.Geometry object from the coordinates
        polygon = ee.Geometry.MultiPolygon(coordinates)

        # Set the maxError parameter
        max_error = 1  # Specify your desired maxError value

        # Project the geometry coordinates
        projected_polygon = polygon.transform('EPSG:4326', max_error)

        #decrease scale to get more rows of data/granularity
        ac = dc.getRegion(projected_polygon, scale=100 ).getInfo()
        df_constructor =pd.DataFrame(ac)

        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['Contact'] = mines_exploded['Contact'].values[i]
        df = df.append(df_constructor)

        print(mines_exploded['Contact'].values[i])
        i+=1

<ipython-input-27-cf6c1c3d415f>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


KY Division of Mine Permits


<ipython-input-27-cf6c1c3d415f>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


KY Division of Mine Permits


<ipython-input-27-cf6c1c3d415f>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


...

<ipython-input-27-cf6c1c3d415f>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


KY Division of Mine Permits


<ipython-input-27-cf6c1c3d415f>:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_constructor)


KY Division of Mine Permits


IndexError: ignored

In [28]:
df

,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice,label,Contact
1,20210104T162701_20210104T162655_T16SGG,-83.527601,37.349704,1609777985993,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits
2,20210104T162701_20210104T162655_T17SKB,-83.527601,37.349704,1609777984728,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits
3,20210114T162631_20210114T162630_T16SGG,-83.527601,37.349704,1610641986404,0.035662,0.194476,0.118468,0.032342,0.046296,0.309017,0.030392,0.145298,0.036263,1,KY Division of Mine Permits
4,20210114T162631_20210114T162630_T17SKB,-83.527601,37.349704,1610641985137,0.041117,0.204949,0.114325,0.038075,0.038577,0.29974,0.032101,0.141967,0.034774,1,KY Division of Mine Permits
5,20210104T162701_20210104T162655_T16SGG,-83.528499,37.350602,1609777985993,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,20210114T162631_20210114T162630_T17SLB,-82.610421,37.770115,1610641980827,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits
9,20210114T162631_20210114T162630_T17SLB,-82.609523,37.770115,1610641980827,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits
10,20210114T162631_20210114T162630_T17SLB,-82.608624,37.770115,1610641980827,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits
11,20210114T162631_20210114T162630_T17SLB,-82.611319,37.771014,1610641980827,None,None,None,None,None,None,None,None,None,None,KY Division of Mine Permits


In [ ]:
#https://stackoverflow.com/questions/55659835/trying-to-separate-polygon-data-into-x-and-y-coordinates-but-get-error-multip

import geopandas as gpd
from shapely.geometry import Polygon, LineString, MultiLineString

def getPolyCoords(row, geom, coord_type):
    """
    Returns the coordinates ('x|y') of edges/vertices of a Polygon/others
    Args:
    - row: the row object from a geodataframe; i.e.   df.loc[1]
    - geom: the name of "geometry" column, usually "geometry"
    - coord_type: 'x', or 'y'
    ...
    Valid geometrie types 'Polygon', 'Point', 'LineString', 'MultiLineString', 'MultiPolygon'
    """

    # Parse the geometries and grab the coordinate
    geometry = row[geom]
    #print(geometry.type)

    if geometry.type=='Polygon':
        if coord_type == 'x':
            # Get the x coordinates of the exterior
            # Interior is more complex: xxx.interiors[0].coords.xy[0]
            return list( geometry.exterior.coords.xy[0] )
        elif coord_type == 'y':
            # Get the y coordinates of the exterior
            return list( geometry.exterior.coords.xy[1] )

    if geometry.type in ['Point', 'LineString']:
        if coord_type == 'x':
            return list( geometry.xy[0] )
        elif coord_type == 'y':
            return list( geometry.xy[1] )

    if geometry.type=='MultiLineString':
        all_xy = []
        # updated code, using .geoms
        for ea in geometry.geoms:
         all_xy = []

In [ ]:
from functools import singledispatch
from itertools import chain
from typing import (List,
                    Tuple,
                    TypeVar)

from shapely.geometry import (GeometryCollection,
                              LinearRing,
                              LineString,
                              Point,
                              Polygon)
from shapely.geometry.base import (BaseGeometry,
                                   BaseMultipartGeometry)

Geometry = TypeVar('Geometry', bound=BaseGeometry)


@singledispatch
def to_coords(geometry: Geometry) -> List[Tuple[float, float]]:
    """Returns a list of unique vertices of a given geometry object."""
    raise NotImplementedError(f"Unsupported Geometry {type(geometry)}")


@to_coords.register
def _(geometry: Point):
    return [(geometry.x, geometry.y)]


@to_coords.register
def _(geometry: LineString):
    return list(geometry.coords)


@to_coords.register
def _(geometry: LinearRing):
    return list(geometry.coords[:-1])


@to_coords.register
def _(geometry: BaseMultipartGeometry):
    return list(set(chain.from_iterable(map(to_coords, geometry))))


@to_coords.register
def _(geometry: Polygon):
    return to_coords(GeometryCollection([geometry.exterior, *geometry.interiors]))

In [17]:
import shapely as sh

def get_coords_from_polygon(shape):
    coords = set()

    if isinstance(shape, sh.geometry.Polygon):
        coords.update(shape.exterior.coords[:-1])
        for linearring in shape.interiors:
            coords.update(linearring.coords[:-1])
    elif isinstance(shape, sh.geometry.MultiPolygon):
        for polygon in shape:
            coords.update(get_coords_from_polygon(polygon))

    return coords

In [18]:
# prompt: get coordinates fro mines_exploded['geometry'].values[i] where it is multipolygon

def get_coords_from_polygon(shape):
    coords = set()

    if isinstance(shape, sh.geometry.Polygon):
        coords.update(shape.exterior.coords[:-1])
        for linearring in shape.interiors:
            coords.update(linearring.coords[:-1])
    elif isinstance(shape, sh.geometry.MultiPolygon):
        for polygon in shape:
            coords.update(get_coords_from_polygon(polygon))

    return coords

mines_geometry = mines_exploded['geometry'].values[i]
coords = get_coords_from_polygon(mines_geometry)


In [ ]:
coordinates

In [ ]:
df

In [ ]:
from google.colab import files
df.to_csv('Kentucky_mines.csv', encoding = 'utf-8-sig')
files.download('Kentucky_mines.csv')